In [3]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

###Basic Generating Restaurant name using lang chain

In [4]:
os.environ['OPENAI_API_KEY']='USe your OPENAI API KEY'
llm = OpenAI(temperature=0.6) 
name=llm("I want to open a fancy indian restaurant can you suggest me a nice name")
print(name)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. Maharaja's Palace
2. Spice Symphony
3. Saffron Garden
4. Royal Tandoor
5. Masala Magic
6. Curry Kingdom
7. The Taj Mahal
8. Namaste India
9. Bollywood Bites
10. Flavors of India
11. Chai and Chaat
12. The Great Indian Feast
13. Tandoori Nights
14. The Royal Spice
15. Curry House 


#####Creating template for generating different name for different type of cuisine

In [4]:
name=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. suggest a fancy name for this."
)
name.format(cuisine="Indian")

'I want to open a restaurant for Indian food. suggest a fancy name for this.'

###Creating single chain for a single prompt to generate the name of a restaurant based on cuisine

In [6]:
chain=LLMChain(llm=llm, prompt=name)
chain.run("chinese")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Dragon\'s Palace"'

### Tried to create 2 chains prompting the name for a restaurant and menu items it should have

In [7]:
llm = OpenAI(temperature=0.6) 
name=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. suggest a fancy name for this."
)
name_chain=LLMChain(llm=llm, prompt=name)

items=PromptTemplate(
    input_variables=['restaurant_name'],
    template="suggest some menu items for r{estaurant_name}. Return it as comma seprated list."
)
food_items=LLMChain(llm=llm,prompt=items)


####For merging 2 or more chains there are many different functions i have tried 2 functions
1.Simple Sequential chain
2.Sequential chain

In [9]:
chain=SimpleSequentialChain(chains=[name_chain,food_items])
chain.run("Italian")

'\n\n1. Caprese Salad: Fresh mozzarella, tomatoes, and basil drizzled with balsamic glaze\n2. Fettuccine Alfredo: Creamy alfredo sauce served over homemade fettuccine\n3. Chicken Parmigiana: Breaded chicken breast topped with marinara sauce and melted mozzarella cheese\n4. Seafood Linguine: Shrimp, scallops, and clams tossed in a white wine garlic sauce\n5. Eggplant Parmigiana: Breaded eggplant topped with marinara sauce and melted mozzarella cheese\n6. Lasagna Bolognese: Layers of homemade pasta, beef ragu, and creamy béchamel sauce\n7. Gnocchi Pesto: Potato dumplings in a creamy pesto sauce with cherry tomatoes and pine nuts\n8. Shrimp Scampi: Sautéed shrimp in a garlic butter sauce served over linguine\n9. Chicken Marsala: Sautéed chicken breast in a marsala wine sauce with mushrooms\n10. Tiramisu: Classic Italian dessert made with ladyfingers, mascarpone cheese, and espresso.'

####Method 2 Sequential chain

In [10]:
llm = OpenAI(temperature=0.6) 
name=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. suggest a fancy name for this."
)
name_chain=LLMChain(llm=llm, prompt=name, output_key="name")

items=PromptTemplate(
    input_variables=['name'],
    template="suggest some menu items for {name}. Return it as comma seprated list."
)
food_items=LLMChain(llm=llm,prompt=items,output_key="Menu")

In [11]:
chain=SequentialChain(chains=[name_chain,food_items],
               input_variables=['cuisine'],output_variables=['name','Menu'])
chain.invoke({'cuisine':'Indian'})

{'cuisine': 'Indian',
 'name': '\n\n"Spice Palace" ',
 'Menu': '\n\n1. Tandoori Chicken\n2. Butter Chicken\n3. Lamb Vindaloo\n4. Vegetable Samosas\n5. Chicken Tikka Masala\n6. Palak Paneer\n7. Chana Masala\n8. Biryani\n9. Garlic Naan\n10. Mango Lassi\n\nTandoori Chicken, Butter Chicken, Lamb Vindaloo, Vegetable Samosas, Chicken Tikka Masala, Palak Paneer, Chana Masala, Biryani, Garlic Naan, Mango Lassi'}

#########In Langchain there are some functions like tools , agents etc as free version of Chatgpt has data upto september 2022 so for getting other information or doing mathematical operation this tools and agents are used
i have tried tools like wikipedia,google search and llm-math.

In [13]:
tools=load_tools(['wikipedia','llm-math'],llm=llm)
agent=initialize_agent(tools,llm,AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)
agent.run("When was Elon musk born? what is his age in 2024")



> Entering new AgentExecutor chain...
 We can use Wikipedia to find out when Elon Musk was born and his current age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: We can use Wikipedia to find out when Elon Musk was born and his current age.
Action: Calculator
Action Input: 2024 - 1971
Observation: Answer: 53
Thought: We now know that Elon Musk will be 53 years old in 2024.
Final Answer: Elon Musk will be 53 years old in 2024.

> Finished chain.


'Elon Musk will be 53 years old in 2024.'

########For working with google api we need to create an account on serp

In [14]:
os.environ['SERPAPI_API_KEY']='Use your own SERP-API Key'
tools=load_tools(['serpapi','llm-math'],llm=llm)
agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,Verbose=True)
agent.run("what was the GDP of US in 2023 ?")

'$27.36 trillion'

########## Here i have tried to work with memory as we have seen in chat-gpt it remembers the previous conversation which we have done so we can also do it using ConversationBufferMemory

In [17]:
memory=ConversationBufferMemory()
chain=LLMChain(llm=llm,prompt=name, memory=memory)
name_1=chain.run("Mexican")
print(name_1)



"El Sabor de México"  (The Flavor of Mexico)


In [18]:
name_2=chain.run("Indian")
print(name_2)



"Spice Palace"


In [21]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor de México"  (The Flavor of Mexico)
Human: Indian
AI: 

"Spice Palace"


######## Conversation buffer memory has some disadvantage as it grows along with the conversation endlessly which is not feasable as memory point of view as well as cost point so to overcome this disadvantage there is a library called ConversationBufferWindowMemory

### Model with rembering only one conversation

In [31]:
memory=ConversationBufferWindowMemory(k=1)
convo=ConversationChain(llm=OpenAI(temperature=0.6),memory=memory)
convo.run("who won the first cricket world cup")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [32]:
convo.run("what is 5+5?")

'  5+5 is equal to 10.'

In [33]:
convo.run("who was the captain of wining team")

' I cannot answer that question as I do not have enough context. Do you have a specific team or event in mind?'

Model with rembering 2 conversation

In [34]:
memory=ConversationBufferWindowMemory(k=2)
convo=ConversationChain(llm=OpenAI(temperature=0.6),memory=memory)
convo.run("who won the first cricket world cup")

' The first Cricket World Cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. The tournament was held in England and was officially called the Prudential Cup. Do you have any other questions about the Cricket World Cup?'

In [35]:
convo.run("what is 5+5?")

' The answer to 5+5 is 10. Would you like to know more about mathematical operations?'

In [36]:
convo.run("who was the captain of wining team")

' The captain of the winning West Indies team in the first Cricket World Cup was Clive Lloyd. He was a left-handed batsman and a medium-paced bowler. He also led the West Indies to victory in the second Cricket World Cup in 1979. Do you have any other questions about the first Cricket World Cup?'